In [32]:
#USING RANDOM GRID SEARCH
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import RobustScaler
# Read the data
df = pd.read_csv('housing-classification-iter6.csv')
df.head()

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [33]:
y = df.pop('Expensive')
X = df

In [34]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
num_cols = X_train.select_dtypes('number').columns
cat_cols = X_train.select_dtypes('object').columns

In [36]:
num_pipe = make_pipeline(SimpleImputer())
cat_pipe = make_pipeline(OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore'))

# Combine the preprocessing pipelines in a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_pipe', num_pipe, num_cols),
        ('cat_pipe', cat_pipe, cat_cols),
    ]
)

# Create the full pipeline with the preprocessing and the model
#full_pipe = make_pipeline(preprocessing, DecisionTreeClassifier())
rfc_full_pipeline = make_pipeline(preprocessor,
                                   RobustScaler(),
                                   RandomForestClassifier()
                                 )

In [52]:
rfc_param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
     'randomforestclassifier__n_estimators':range(5, 60, 2),
     'randomforestclassifier__min_samples_split': range (2, 10, 2),
     'randomforestclassifier__min_samples_leaf': range (5, 15, 2),
     'randomforestclassifier__max_depth': None,
    
}

random_rfc_search = RandomizedSearchCV(rfc_full_pipeline,
                      rfc_param_grid,
                      cv=5,
                      verbose=1)

random_rfc_search.fit(X_train, y_train)

random_rfc_search.best_score_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. 

C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [17, 21, 25, 30, 32] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [7, 25, 26, 27, 31] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its def

C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.9469058361762224

In [53]:
random_rfc_search.best_params_

{'randomforestclassifier__n_estimators': 59,
 'randomforestclassifier__min_samples_split': 2,
 'randomforestclassifier__min_samples_leaf': 5,
 'randomforestclassifier__max_depth': None,
 'columntransformer__num_pipe__simpleimputer__strategy': 'mean'}

In [54]:
pred_train = random_rfc_search.predict(X_train)
accuracy_score(y_train, pred_train)

0.9708904109589042

In [55]:
pred_train = random_rfc_search.predict(X_train)
pred_test = random_rfc_search.predict(X_test)

C:\Users\leoos\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [25, 31] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [56]:
print('Train Acc = ', accuracy_score(y_train, pred_train))
print('Test Acc = ', accuracy_score(y_test, pred_test))

Train Acc =  0.9708904109589042
Test Acc =  0.952054794520548
